# Instructions
The following code was designed in order to load in and score motion/freezing data from video files. This code should be run with carefully selected parameters using the more extensive visualization options provided in FreezeAnalyis.ipynb.  In addition to saving frame by frame motion/freezing information for each video in separate csv files, bins can also be defined for summarizing videos (e.g. minute by minute).  All summary information will be saved in a single file.  If no bins are defined, the code will automatically create a summary file that lists average session motion/freezing per video.

### Package Requirements
The following will need to be installed in your Conda environment:

- python (3.6.5)

- jupyter

- imread

- mahotas(1.4.4)

- numpy(1.14.3)

- pandas(0.23.0)

- matplotlib(2.2.2). 

- opencv(3.4.3

The following commands can be executed in your terminal to create an environment with these packages: 

- conda config --add channels conda-forge

- conda create -n EnvironmentName python=3.6.5 mahotas=1.4.4 pandas=0.23.0 matplotlib=2.2.2 opencv=3.4.3 jupyter imread

# 1. Load Necessary Packages

In [ ]:
import pylab 
import os
import cv2
import fnmatch
import numpy as np
import mahotas as mh 
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# 2. User Sets Directory and Parameters for Motion/Freeze Analysis
Note that all videos of particular type will be analyzed.  Videos should be of the same length if user is defining bins.  Videos should be the same fps and the same width/height.

In [ ]:
# directory containing all video files to be processed
dpath = "/Users/ZP/Videos" 

#video parameters
ftype = "mpg" #file type.  so far only mpg1 ("mpg") and wmv ("wmb") have been tested
fps = 30 #fps of video files
ycrop = 140 #cropping of video frame
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 

#Motion/Freezing parameters
mt_cutoff = 10.3 #Motion cutoff
FreezeThresh = 100 #Upper bound for freezing, in frame-by-frame pixels changed
MinDuration = 1 #Number of seconds motion must be below threshold to beging accruing freezing=

# 3. User Defines Bins for Summary data

In [ ]:
Use_Bins = False #Define whether user specified summary bins are to bew used.  True/False.

#USER SETS BIN INFORMATION
Bin_Names = ['avg','1','2','3','4','5'] #Create list of bin names. Must be in single/double quotes (eg: ['avg','1','2'])
Bin_Start = [0,0,60,120,180,240] #provide list of bin start times, in seconds (eg: [0,0,60])
Bin_Stop = [300,60,120,180,240,300] #provide list of bin end times, in seconds (eg: [120,60,120])

#Check requirement that lists be equal length
if len(Bin_Names)!=len(Bin_Start)!=len(Bin_Stop):
    print('WARNING.  Bin list sizes are not of equal length')    

# 4. Load Functions

In [ ]:
def Measure_Motion(fpath):
    
    #Upoad file
    cap = cv2.VideoCapture(fpath)

    #Get maxiumum frame of file. Note that this is updated later if fewer frames detected
    cap_max = int(cap.get(7)) #7 is index of total frames

    #Set first frame to be grabbed
    cap.set(1,0) #first index references frame property, second specifies next frame to grab

    #Initialize first frame
    ret, frame = cap.read()
    frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_new = frame_new[ycrop:,:]
    frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA)
    
    #Initialize vector to store motion values in
    Motion = np.zeros(cap_max)

    #Loop through frames to detect frame by frame differences
    for x in range (1,cap_max):

        #Reset old frame
        frame_old = frame_new

        #Attempt to load next frame
        ret, frame = cap.read()
        if ret == True:

            #Reset new frame and process
            frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_new = frame_new[ycrop:,:]
            frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA) # used to reduce influence of jitter from one frame to the next

            #Calculate difference between frames
            frame_dif = np.absolute(frame_new - frame_old)
            frame_cut = frame_dif > mt_cutoff
            frame_cut = frame_cut.astype('uint8')

            #Assign difference to array
            Motion[x]=np.sum(frame_cut)

        else: 
            #if no frame is detected
            cap_max = (x-1) #Reset max frame to last frame detected
            Motion = Motion[:cap_max] #Amend length of motion vector
            break
        
    print('total frames: ' + str(cap_max))

    #release video
    cap.release() 
    
    #return motion values
    return(Motion)

def Measure_Freezing(Motion):

    #Find frames below thresh
    BelowThresh = (Motion<FreezeThresh).astype(int)

    #Perform local cumulative thresh detection
    #For each consecutive frame motion is below threshold count is increased by 1 until motion goes above thresh, at which point coint is set back to 0
    CumThresh = np.zeros(len(Motion))
    for x in range (1,len(Motion)):
        if (BelowThresh[x]==1):
            CumThresh[x] = CumThresh[x-1] + BelowThresh[x]

    #Measure Freezing
    Freezing = (CumThresh>=MinDuration).astype(int) #whenever motion has dropped below the thresh for at least MinDuration call this freezing
    
    #the following code makes it so that the initial 30 before CumThresh meets MinDuration are still counted as freezing
    #Debating whether the code renders better results
    for x in range( len(Freezing) - 2, -1, -1) : 
        if Freezing[x] == 0 and Freezing[x+1]>0 and Freezing[x+1]<MinDuration:
            Freezing[x] = Freezing[x+1] + 1
    Freezing = (Freezing>0).astype(int)
    
    return(Freezing)

def SaveData(file,fpath,Motion,Freezing):
    
    #Set output name
    fpath_out = fpath[:-4] + '_FreezingOutput.csv'

    #Create Dataframe
    DataFrame = pd.DataFrame(
        {'File': [file]*len(Motion),
         'FPS': np.ones(len(Motion))*fps,
         'MotionCutoff':np.ones(len(Motion))*mt_cutoff,
         'FreezeThresh':np.ones(len(Motion))*FreezeThresh,
         'MinFreezeDuration':np.ones(len(Motion))*MinDuration,
         'Frame': np.arange(len(Motion)),
         'Motion': Motion,
         'Freezing': Freezing
        })   

    DataFrame.to_csv(fpath_out)
    
    
def Summarize(file,Motion,Freezing):
    
    #Initialize arrays to store summary values in
    mt = np.zeros(len(Bin_Names)) 
    fz = np.zeros(len(Bin_Names)) 
    
    #Get averages for each bin
    for Bin in range (len(Bin_Names)):
        if len(Motion)<Bin_Stop[Bin]: #if end of video falls within bin, truncate bin to match video length
            mt[Bin]=np.mean(Motion[Bin_Start[Bin] : len(Motion)])
            fz[Bin]=(np.mean(Freezing[Bin_Start[Bin] : len(Motion)]))*100
        else:
            mt[Bin]=np.mean(Motion[Bin_Start[Bin] : Bin_Stop[Bin]])
            fz[Bin]=(np.mean(Freezing[Bin_Start[Bin] : Bin_Stop[Bin]]))*100
            
    #Create data frame to store data in
    df = pd.DataFrame(
    {'File': [file]*len(Bin_Names),
     'FileLength': np.ones(len(Bin_Names))*len(Motion),
     'FPS': np.ones(len(Bin_Names))*fps,
     'MotionCutoff':np.ones(len(Bin_Names))*mt_cutoff,
     'FreezeThresh':np.ones(len(Bin_Names))*FreezeThresh,
     'MinFreezeDuration':np.ones(len(Bin_Names))*MinDuration,
     'Bin': Bin_Names,
     'Bin_Start(f)': Bin_Start,
     'Bin_Stop(f)': Bin_Stop,
     'Motion': mt,
     'Freezing': fz
    })   
    
    return(df)
     
    

# 5. Analyze Videos

In [ ]:
#Convert necessary parameters from seconds to frames
MinDuration = MinDuration * fps 
Bin_Start = [x * fps for x in Bin_Start]
Bin_Stop = [x * fps for x in Bin_Stop]

#Get list of video files
if os.path.isdir(dpath):
    FileNames = sorted(os.listdir(dpath))
    FileNames = fnmatch.filter(FileNames, ('*.' + ftype)) #restrict files to .mpg videos
else:
    print('Directory not found. Check that path is correct.')

#Loop through files    
for file in FileNames:
    
    #Set file
    fpath = dpath + "/" + file
    print('Processing: ' + file)
    
    #Analyze frame by frame motion and freezing and save csv of results
    Motion = Measure_Motion(fpath)
    Freezing = Measure_Freezing(Motion)  
    SaveData(file,fpath,Motion,Freezing)
    
    #Create summary info for file, either based upon user-defined bins or taking average of entire video
    if Use_Bins == True:
        if len(Motion)<max(Bin_Start):
            print('Bin parameters exceed length of video.  Cannot create summary')
        elif len(Motion)>max(Bin_Start):
            summary = Summarize(file,Motion,Freezing)
    elif Use_Bins == False:
        Bin_Names = ['avg'] 
        Bin_Start = [0] 
        Bin_Stop = [len(Motion)] 
        summary = Summarize(file,Motion,Freezing)
    
    #Add summary info for individual file to larger summary of all files
    try:
        summary_all = pd.concat([summary_all,summary])
    except NameError: #to be done for first file in list, before summary_all is created
        summary_all = summary

#Write summary data to csv file
sumpath_out = dpath + "/" + 'Summary.csv'
summary_all.to_csv(sumpath_out)